In [11]:
import os

In [12]:
%pwd

'/home/vaibhav/Chest_Cancer_Classification_Using_MLFlow-DVC'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/vaibhav/Chest_Cancer_Classification_Using_MLFlow-DVC'

In [13]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list
    early_stopping_patience: int

In [14]:
# Update the configuration manager in src config
from Chest_Cancer_Classification.constants import *
from Chest_Cancer_Classification.utils.common import read_yaml,create_directories
import tensorflow as tf

In [15]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_roots])

        

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "chest_cancer_img")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE,
            early_stopping_patience=params.early_stopping_patience
        )

        return training_config

In [16]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time
from tensorflow.keras.callbacks import EarlyStopping

In [19]:


class Training:
    def __init__(self, config):
        self.config = config
        self.model = None
        self.train_generator = None
        self.valid_generator = None

    def get_base_model(self):
        self.model = tf.keras.models.load_model(self.config.updated_base_model_path)

    def train_valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        # For validation data
        validation_data_dir = Path(self.config.training_data)  / 'Data'/'test'
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=str(validation_data_dir),
            shuffle=False,
            **dataflow_kwargs
        )

        # For training data
        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        training_data_dir = Path(self.config.training_data)  /'Data' /'train'
        self.train_generator = train_datagenerator.flow_from_directory(
            directory=str(training_data_dir),
            shuffle=True,
            **dataflow_kwargs
        )
        print(self.config.training_data / 'train')

    @staticmethod
    def save_model(path, model):
        model.save(path)

    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        early_stopping = EarlyStopping(
            monitor='val_loss',  # or another metric you want to monitor
            patience=self.config.early_stopping_patience,
            restore_best_weights=True
        )

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=[early_stopping]
        )

        self.save_model(path=self.config.trained_model_path, model=self.model)

       

In [20]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2024-02-27 12:30:13,141: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-02-27 12:30:13,144: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-27 12:30:13,145: INFO: common: created directory at: artifacts]
[2024-02-27 12:30:13,145: INFO: common: created directory at: artifacts/training]
Found 174 images belonging to 2 classes.
Found 343 images belonging to 2 classes.
artifacts/data_ingestion/chest_cancer_img/train
Epoch 1/10


2024-02-27 12:30:13.885747: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


 4/21 [====>.........................] - ETA: 39s - loss: 11.7600 - accuracy: 0.5312